In [235]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from multiprocessing import Process, Queue
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
import pandas as pd
from scipy.misc import imsave
import scipy.stats
import random
from sklearn.metrics import accuracy_score

from problog import get_evaluatable
from problog.program import PrologString, SimpleProgram
from problog.core import ProbLog
from problog.logic import Constant,Var,Term,AnnotatedDisjunction
from problog.learning import lfi

%matplotlib notebook

In [232]:
st1 = """
st1 += "t(0.5)::gender_male;\n0.5::gender_female.\n"
st1 += "t(0.25)::age_1;\n0.25::age_2;\n0.25::age_3;\n0.25::age_4.\n"
"""

# st1 += "0.5::alcohol.\n"
# st1 += "0.5::smoke.\n"
# st1 += "0.5::blood_pressure.\n"
# st1 += "0.5::ldl.\n"
# st1 += "0.5::bmi.\n"
# st1 += "0.5::heart_d.\n"
# st1 += "0.5::glucose.\n"


In [233]:
st2 = ""
st2 += "t(0.5)::blood_pressure:-gender_male.\n"
st2 += "t(0.5)::blood_pressure:-gender_female.\n"

# st2 += "0.25::blood_pressure:-age_1.\n"
# st2 += "0.25::blood_pressure:-age_2.\n"
# st2 += "0.25::blood_pressure:-age_3.\n"
# st2 += "0.25::blood_pressure:-age_4.\n"

# st2 += "t(-)::blood_pressure:-alcohol.\n"
# st2 += "t(-)::blood_pressure:-smoke.\n"

In [268]:
model = """
t(0.5)::gender_male;t(0.5)::gender_female.
t(0.25)::age_1;t(0.25)::age_2;t(0.25)::age_3;t(0.25)::age_4.


t(0.5)::blood_pressure:-gender_male.
t(0.5)::blood_pressure:-gender_female.

t(_)::blood_pressure:-age_1.
t(_)::blood_pressure:-age_2.
t(_)::blood_pressure:-age_3.
t(_)::blood_pressure:-age_4.


"""

In [269]:
gender_male = Term('gender_male')
gender_female = Term('gender_female')

age_1 = Term('age_1')
age_2 = Term('age_2')
age_3 = Term('age_3')
age_4 = Term('age_4')



blood_pressure = Term('blood_pressure')

In [270]:
examples = [
    [(gender_female, True), (age_2, True), (blood_pressure, True)],
    [(gender_female, False), (age_2, False), (blood_pressure, False)]
]

In [271]:
score, weights, atoms, iteration, lfi_problem = lfi.run_lfi(PrologString(model), examples)

In [272]:
print (lfi_problem.get_model())

0.5::gender_male; 0.5::gender_female.
8.747042536e-06::age_1; 0.5::age_2; 0.0::age_3; 0.499990750635133::age_4.
0.0::blood_pressure :- gender_male.
1.0::blood_pressure :- gender_female.
0.00237871623847::blood_pressure :- age_1.
0.667350415482241::blood_pressure :- age_2.
0.130627868887896::blood_pressure :- age_3.
0.0::blood_pressure :- age_4.

